In [16]:
from openai import AzureOpenAI
import time
import bot as bot
from eval_utils import *
import re

import warnings
warnings.filterwarnings("ignore", message="CropBox missing from /Page") 
warnings.filterwarnings("ignore", message=".*ScriptRunContext.*")

**Extract Q&A from file**

In [18]:
qa_pairs = extract_qa_pairs_from_pdf('./docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


# FidChat

In [24]:
instructions = bot.instructions

In [25]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint="https://ai-bcds.openai.azure.com/",
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o-mini", 
    instructions = instructions,
    tool_resources={"code_interpreter":{"file_ids":["assistant-WZQHoFwobKHT1ZWKkEAGqm","assistant-MsQgxjsadXVcZKLw2fJwNh","assistant-LpaoUAR5Rq3MgXfMSU4i9A","assistant-KW358eNBGny1N5DM8LoBaC","assistant-CiKqiv2zmbtF8FH2iQAKAw","assistant-9yhV7cKx5ynoLqoPbs1B67","assistant-5QanyuMMkjkkfVUv6rkCfM"]}},
    temperature=0.2,
    top_p=1
    )

In [26]:
def fidchat(client, assistant_id, message, instructions):
    # Create a new thread
    thread = client.beta.threads.create()

    # Add message (with optional system instructions)
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=instructions + "\n\n" + message
    )

    # Run assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait until response is ready
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Return the assistant's response
    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                raw_response = msg.content[0].text.value
                clean_response = re.sub(r"【\d+:\d+†source】", "", raw_response).strip()
                return clean_response
    return "[ERROR] Failed to get assistant response."


# EvalChat

In [27]:
client2 = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
  api_version="2024-05-01-preview"
)

assistant2 = client.beta.assistants.create(
  model="gpt-4o-mini-BCwDS", 
  instructions="",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_ezpssNVZDpwb0RjgcxSwbNZz"]}},
  temperature=1,
  top_p=1
)

In [40]:
def eval_chat(client, assistant_id, question, expected_answer, instructions):
    # Create a new thread
    thread = client.beta.threads.create()

    # Add message with instructions and the provided answer to evaluate
    prompt = f"{instructions}\n\nQuestion: {question}\nAnswer: {expected_answer}"
    
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    # Run assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait for response
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                raw_response = msg.content[0].text.value
                clean_response = re.sub(r"【\d+:\d+†source】", "", raw_response).strip()
                return clean_response
    return "[ERROR] Failed to get assistant response."


# Evaluation

In [54]:
# qa_pairs
instructions2 = 'Give a similarity score between expected answer and actual answer. \
                 Output should be a number between 0-10. Do not write anything else'

In [57]:
results = []

for idx, (question, expected_answer) in enumerate(qa_pairs):
    print(f"Evaluating Q{idx+1}...")
    
    # Get assistant's answer
    actual_answer = fidchat(client, assistant.id, question, instructions)
    evaluation = eval_chat(client2, assistant2.id, question, actual_answer, instructions2)

    results.append({
        "question": question,
        "expected_answer": expected_answer,
        "actual_answer": actual_answer,
        "evaluation": evaluation
    })
    


Evaluating Q1...
Evaluating Q2...
Evaluating Q3...
Evaluating Q4...
Evaluating Q5...
Evaluating Q6...
Evaluating Q7...
Evaluating Q8...
Evaluating Q9...
Evaluating Q10...
Evaluating Q11...
Evaluating Q12...
Evaluating Q13...
Evaluating Q14...
Evaluating Q15...
Evaluating Q16...
Evaluating Q17...


In [58]:
for r in results:
    print("\n---")
    print("Q:", r["question"])
    print('')
    print("EA:", r["expected_answer"])
    print('')
    print("AA:", r["actual_answer"])
    print('')
    print("Evaluation:", r["evaluation"])


---
Q: O Fidelidade Savings é mais indicado para que perfil de risco?

EA: o Fidelidade Savings é adequado para clientes conservadores, através da sua opção Seguro, com capital e rendimento garantido. Para clientes que gostam moderadamente do risco, através da opção Proteção com garantia de 90% do capital investido no vencimento e exposição ao Índice Multiativos. Para clientes propensos ao risco através da opção Dinâmico com 100% de exposição ao Índice Multiativos. Também está disponível a opção ESG, que investe no índice Multiativos ESG, com risco mas ESG, para clientes com preocupações Ambientais, Sociais e de Governança.

AA: O **Fidelidade My Savings** é um produto de poupança que se destina a investidores com um perfil de risco **baixo a moderado**. Este produto é ideal para aqueles que procuram uma forma segura de poupança, com a possibilidade de obter rendimentos, mas que não desejam expor-se a riscos elevados associados a investimentos mais voláteis.

Se tiveres mais perguntas

In [75]:
for i in range(len(results)):
    print(f'Q{i+1}', results[i]["evaluation"])


Q1 7
Q2 7
Q3 7
Q4 7
Q5 8
Q6 5
Q7 8
Q8 8
Q9 8
Q10 9
Q11 3
Q12 10
Q13 9
Q14 7
Q15 9
Q16 7
Q17 8
